In [69]:
import numpy as np
import cvxpy as cp

In [70]:
def convert_accounts(accounts):
    converted_accounts = {}
    for account, values in accounts.items():
        sorted_keys = sorted(values.keys())
        intervals = []
        for i, key in enumerate(sorted_keys):
            start = key
            end = sorted_keys[i + 1] if i + 1 < len(sorted_keys) else float('inf')
            intervals.append((start, end, values[key]))
        converted_accounts[account] = intervals
    return converted_accounts

In [75]:
# Savings accounts with progressive interest rates

banks = dict()

banks['LH'] = {
    0.0: 0.0025,
    1e5: 0.0175,
    9e5: 0.0555,
    1e6: 0.015,
    3e6: 0.0025,
    1e8: 0.0,
}

banks['KKP'] = {
    0.0: 0.02,
    5e3: 0.04,
    1e4: 0.02,
    5e4: 0.0155,
    1.5e6: 0.005,
}

banks['Dime'] = {
    0.0: 0.03,
    1e4: 0.015,
    1e6: 0.005,
}

banks['Chill-D'] = {
    0.0: 0.005,
    1e4: 0.018,
    5e4: 0.0288,
    1e5: 0.002,
}

banks['Kept'] = {
    0.0: 0.0187,
}

banks['TTB'] = {
    0.0: 0.022,
    1e5: 0.016,
    1e6: 0.012,
}

banks['Thai Credit'] = {
    0.0: 0.02,
    5e5: 0.007,
}

banks['Speed D+'] = {
    0.0: 0.008,
    1e5: 0.0188,
    2e7: 0.0,
}

accounts = convert_accounts(banks)

In [93]:
# Total investment amount

total_investment = 10000000

In [94]:
# Create investment variables for each segment in each account
investment_vars = {}
indicator_vars = {}
constraints = []

# Create investment variables for each segment in each account
investment_vars = {}
indicator_vars = {}
constraints = []

for account, segments in accounts.items():
    investment_vars[account] = []
    indicator_vars[account] = []
    for i, (lower, upper, rate) in enumerate(segments):
        var = cp.Variable(nonneg=True)
        indicator = cp.Variable(boolean=True)
        investment_vars[account].append(var)
        indicator_vars[account].append(indicator)
        
        upper_bound = upper if upper != float('inf') else total_investment
        if i == 0:
            constraints.append(var <= upper_bound * indicator)
        else:
            constraints.append(var <= (upper_bound - segments[i-1][1]) * indicator)

# Objective function: Maximize total return
total_return = cp.sum([
    investment_vars[account][i] * rate
    for account, segments in accounts.items()
    for i, (lower, upper, rate) in enumerate(segments)
])

# Constraints
for account, segments in accounts.items():
    for i, (lower, upper, rate) in enumerate(segments):
        upper_bound = upper if upper != float('inf') else total_investment
        if i == 0:
            constraints.append(investment_vars[account][i] <= upper_bound)
        else:
            constraints.append(investment_vars[account][i] <= (upper_bound - lower) * indicator_vars[account][i])
            constraints.append(cp.sum(investment_vars[account][:i+1]) <= upper_bound)

# Constraint: Total investment must not exceed the available amount
constraints.append(cp.sum([var for account in investment_vars.values() for var in account]) <= total_investment)

# Ensure that if a segment is invested in, all previous segments must be fully invested
for account, segments in accounts.items():
    for i in range(1, len(segments)):
        constraints.append(indicator_vars[account][i] <= indicator_vars[account][i-1])
        constraints.append(investment_vars[account][i-1] >= (segments[i-1][1] - segments[i-1][0]) * indicator_vars[account][i])

# Define the optimization problem
problem = cp.Problem(cp.Maximize(total_return), constraints)

# Solve the problem
problem.solve(solver=cp.GLPK_MI)

# Print the results
print(f"Initial Fund: {total_investment}")

print("="*20)

print("Investment Strategy:")

for account in accounts:
    print(f"{account}: {np.sum([investment_vars[account][i].value for i in range(len(accounts[account]))]):.2f}")

# for account in accounts:
#     for i in range(len(accounts[account])):
#         print(f"{account}_Segment_{i}: {investment_vars[account][i].value:.2f}")

print("="*20)

print(f"Total Return: {total_return.value:.2f}")

Initial Fund: 10000000
Investment Strategy:
LH: 1000000.00
KKP: 50000.00
Dime: 10000.00
Chill-D: 100000.00
Kept: 8240000.00
TTB: 100000.00
Thai Credit: 500000.00
Speed D+: 0.00
Total Return: 189698.00
